# Run MinION with Guppy 

- This scripts runs guppy minion step by step. The actual run can be found in run_minion.py

In [1]:
# Import all packages

import sys
sys.path.append("/home/emre/github_repo/MinION")

from minION.util import IO_processor
from minION import analyser
from minION import consensus
from minION import demultiplexer
from minION.util import globals



import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import numpy as np
from Bio import SeqIO
import matplotlib.pyplot as plt
import gzip



In [2]:
# General settings

result_path = Path("/home/emre/")
experiment_name = "20231130_RL-5sites-8plates_flongle"

# Add conditions to avoid running the script accidentally
skip_basecalling = True
skip_demultiplex = False



### Step 1 (Optional): Basecall reads

- Basecall can usually be done while sequencing (if GPU available?)
- Otherwise, basecall afterwards

In [3]:
# Path to pod5 files
#file_to_pod5 = "/var/lib/minknow/data/20230905_errorprone-3_test/no_sample/20230905_1342_MN41105_flg114_5c170bfa/pod5/"
file_to_fastq = "/var/lib/minknow/data/20231130_RL-5sites-8plates_flongle/no_sample/20231130_1900_MN41105_FLG114_1904f884/fastq_pass/"
basecall_model_type = "sup"
basecall_model = globals.DORADO_MODELS[basecall_model_type]
output_name = experiment_name


result_folder = IO_processor.create_folder( experiment_name,
                                            basecall_model_type, 
                                            target_path=result_path)
                                            
experiment_folder = IO_processor.find_experiment_folder(experiment_name) # Folder where pod5 files are located

basecall_folder = result_folder / "basecalled_filtered"
basecall_folder.mkdir(parents=True, exist_ok=True)

print(basecall_folder)
if not skip_basecalling:
    pod5_files = IO_processor.find_folder(experiment_folder, "pod5")
    run_dorado(basecall_model, pod5_files, basecall_folder, fastq = True)


/home/emre/minION_results/20231130_RL-5sites-8plates_flongle_sup/basecalled_filtered


### Step 2: Demultiplex Barcodes using guppy barcoder

- Define the barcodes in the barcoding folder and index them correctly
- We first run reverse barcodes and split in to # RBC folders. Within each RBC folder, guppy barcoder is run again for front barcodes


In [ ]:
if not skip_demultiplex:

    BARCODES = {"Barcode-kit-FBC" : "FBC-v1",
                "Barcode-kit-RBC" : "RBC-v1"}

    output_folder_name = "demultiplex_50_guppy"

    demultiplexer.run_demultiplexer(result_folder, BARCODES, 50, 50, output_folder_name=output_folder_name, basecall_folder = basecall_folder)

### Step 3: Get consensus sequence with guppy

In [3]:
demultiplex_folder = Path("/home/emre/minION_results/MinION_RBC_0902723_sup/Demultiplex_cpp_70_40k_reads")
ref_seq = Path("/home/emre/github_repo/MinION/minION/refseq/hetcpiii_padded.fasta")
barcode_dicts = IO_processor.get_barcode_dict(demultiplex_folder, "NB", "RB")


In [6]:

# analyser.get_variant_df(demultiplex_folder, ref_seq, barcode_dict, consensus_folder_name = "consensus" , sequences = False)

for barcode_id, barcode_dict in barcode_dicts.items():

    rbc = os.path.basename(barcode_id)

    for front_barcode in barcode_dict:

        fbc = os.path.basename(front_barcode)

        consensus.get_consensus(front_barcode, ref_seq, output_name = "consensus.fastq", qualities = True, consensus_folder = "consensus")
        

[21:01:22 - Predict] Reducing threads to 2, anymore is a waste.
[21:01:22 - Predict] Setting tensorflow inter/intra-op threads to 2/1.
[21:01:22 - Predict] Processing region(s): HetCPIII:0-712
[21:01:22 - Predict] Using model: /home/emre/miniconda3/envs/medaka/lib/python3.10/site-packages/medaka/data/r1041_e82_400bps_sup_v4.2.0_model.tar.gz.
[21:01:22 - Predict] Found a GPU.
[21:01:22 - Predict] If cuDNN errors are observed, try setting the environment variable `TF_FORCE_GPU_ALLOW_GROWTH=true`. To explicitely disable use of cuDNN use the commandline option `--disable_cudnn. If OOM (out of memory) errors are found please reduce batch size.
[21:01:22 - Predict] Processing 1 long region(s) with batching.
[21:01:22 - ModelLoad] GPU available: building model with cudnn optimization
[21:01:26 - MdlStrTF] Model <keras.engine.sequential.Sequential object at 0x7f3646fa77c0>
[21:01:26 - MdlStrTF] loading weights from /tmp/tmpj3ba7l91/model/variables/variables
[21:01:26 - BAMFile] Creating pool o